Dalam latihan ini kita akan melihat perbedaan fungsi dan operasi antara RDD dan DataFrame, yaitu filtering dan agregasi.

Instal pyspark

In [1]:
%pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 7.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=91228b199fbaf084cf52b58ba251c5d0b542876e29c8fed8d0c2890e0423c84e
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


Import package yang dibutuhkan

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

Create spark session

In [3]:
spark = SparkSession.builder.appName('RDD & DataFrame').getOrCreate()

## Create 



In [7]:
listData = [("Banten",1222258,"Lebak"),
("Banten",1175148,"Pandeglang"),
("Banten",1435003,"Serang"),
("Banten",2619803,"Tangerang"),
("DKI Jakarta",23340,"Kepulauan Seribu"),
("DKI Jakarta",2185711,"Jakarta Selatan"),
("DKI Jakarta",2843816,"Jakarta Timur"),
("DKI Jakarta",914182,"Jakarta Pusat"),
("DKI Jakarta",2463560,"Jakarta Barat"),
("DKI Jakarta",1747315,"Jakarta Utara")]

listData

[('Banten', 1222258, 'Lebak'),
 ('Banten', 1175148, 'Pandeglang'),
 ('Banten', 1435003, 'Serang'),
 ('Banten', 2619803, 'Tangerang'),
 ('DKI Jakarta', 23340, 'Kepulauan Seribu'),
 ('DKI Jakarta', 2185711, 'Jakarta Selatan'),
 ('DKI Jakarta', 2843816, 'Jakarta Timur'),
 ('DKI Jakarta', 914182, 'Jakarta Pusat'),
 ('DKI Jakarta', 2463560, 'Jakarta Barat'),
 ('DKI Jakarta', 1747315, 'Jakarta Utara')]

### Create RDD

Create spark RDD dengan menggunakan fungsi `sparkContext.parallelize()`

In [8]:
dataRDD = spark.sparkContext.parallelize(listData)

Tampilkan hasilnya dengan perintah `collect`

**Perhatikan** bahwa ketika kita memanggil perintah `collect()` terhadap sebuah RDD ataupun Dataset, **seluruh data dalam RDD tersebut akan dikirim ke node di mana driver berada**. Oleh karena itu perlu dipertimbangkan ukuran data sebelum memanggil fungsi `collect`.

In [9]:
dataRDD.collect()

[('Banten', 1222258, 'Lebak'),
 ('Banten', 1175148, 'Pandeglang'),
 ('Banten', 1435003, 'Serang'),
 ('Banten', 2619803, 'Tangerang'),
 ('DKI Jakarta', 23340, 'Kepulauan Seribu'),
 ('DKI Jakarta', 2185711, 'Jakarta Selatan'),
 ('DKI Jakarta', 2843816, 'Jakarta Timur'),
 ('DKI Jakarta', 914182, 'Jakarta Pusat'),
 ('DKI Jakarta', 2463560, 'Jakarta Barat'),
 ('DKI Jakarta', 1747315, 'Jakarta Utara')]

### Create DataFrame
Create DataFrame dengan menggunakan fungsi `createDataFrame()`

In [10]:
dataDF = spark.createDataFrame(listData, ["province", "pop","district"])


Tampilkan hasilnya dengan perintah `show`

In [11]:
dataDF.show()

+-----------+-------+----------------+
|   province|    pop|        district|
+-----------+-------+----------------+
|     Banten|1222258|           Lebak|
|     Banten|1175148|      Pandeglang|
|     Banten|1435003|          Serang|
|     Banten|2619803|       Tangerang|
|DKI Jakarta|  23340|Kepulauan Seribu|
|DKI Jakarta|2185711| Jakarta Selatan|
|DKI Jakarta|2843816|   Jakarta Timur|
|DKI Jakarta| 914182|   Jakarta Pusat|
|DKI Jakarta|2463560|   Jakarta Barat|
|DKI Jakarta|1747315|   Jakarta Utara|
+-----------+-------+----------------+



### Filter data RDD

Untuk melakukan filtering RDD, kita menggunakan fungsi transformasi `filter`, dengan parameter berupa ekspresi `lambda`.

RDD tidak memiliki skema, sehingga kita mengakses baris dan kolom dengan menggunakan indeks.

Perhatikan bahwa perintah-perintah ini tidak langsung dijalankan oleh Spark, meskipun kita meng-execute cell yang bersangkutan.

In [12]:
rddf = dataRDD.filter(lambda x: x[0] == 'Banten')

In [13]:
rddf.collect()

[('Banten', 1222258, 'Lebak'),
 ('Banten', 1175148, 'Pandeglang'),
 ('Banten', 1435003, 'Serang'),
 ('Banten', 2619803, 'Tangerang')]

### Filtering DataFrame

Untuk melakukan filtering, digunakan fungsi `filter`. 

Karena DataFrame memiliki skema, maka kita dapat menggunakan nama kolom untuk melakukan filtering

In [14]:
dff = dataDF.filter(dataDF['province'] == 'Banten')

Tampilkan hasilnya dengan perintah `show`

In [15]:
dff.show()

+--------+-------+----------+
|province|    pop|  district|
+--------+-------+----------+
|  Banten|1222258|     Lebak|
|  Banten|1175148|Pandeglang|
|  Banten|1435003|    Serang|
|  Banten|2619803| Tangerang|
+--------+-------+----------+



## Group by dan Agregasi 

### RDD

Untuk melakukan group by dan agregasi pada RDD, kita menggunakan fungsi transformasi `map` dan `reduceByKey`, dengan mengirimkan fungsi `lambda` sebagai parameter operasinya.

- `map` menerapkan ekspresi `lambda` pada **setiap element x** RDD 
- `reduceByKey` melakukan operasi agregasi berdasar key tertentu sesuai ekspresi `lambda` 

In [16]:
# map 1       : tambahkan integer 1 di tiap row untuk melakukan count
# reduceByKey : jumlahkan (sum) kolom populasi dan count
# map 2       : hitung rata-rata dengan operasi : sum(populasi)/count

popRDD = (dataRDD  \
           .map(lambda x: (x[0], (x[1], 1)))  \
           .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))  \
           .map(lambda x: (x[0], x[1][0]/x[1][1])))                 

Tampilkan hasilnya

In [17]:
popRDD.collect()

[('DKI Jakarta', 1696320.6666666667), ('Banten', 1613053.0)]

### DataFrame

Groupby dan agregat dengan menggunakan fungsi `groupBy` dan `agg`. 

In [18]:
# Group by province name and average population 
avgDF = dataDF.groupBy("province").agg(avg("pop"))

Tampilkan hasilnya 

In [19]:
# Show the results
avgDF.show()

+-----------+------------------+
|   province|          avg(pop)|
+-----------+------------------+
|DKI Jakarta|1696320.6666666667|
|     Banten|         1613053.0|
+-----------+------------------+



In [ ]:
#spark.sql("SET -v").select("key", "value").show(n=200, truncate=False)

In [ ]:
 #spark.conf.get("spark.sql.shuffle.partitions")

In [ ]:
 #spark.conf.set("spark.sql.shuffle.partitions", 5)

In [ ]:
#spark.conf.get("spark.default.parallelism")

## 

Update history

2301.1729
* _Add filtering_
* _Code cleanup_
* _Add comments & explanations_